This is a brief tutorial of basic Metagraph usage.

First, we import metagraph:

In [ ]:
import metagraph as mg

## Inspecting types and algorithms available

The default resolver automatically pulls in all registered metagraph plugins

In [ ]:
res = mg.resolver

A hierarchy of available types is automatically added as properties on `res`

In [ ]:
dir(res.types)

For each abstract type, there are several concrete types. All concrete types within
that single abstract type represent equivalent data, but in a different format
or data structure.

Here we show the concrete types which represent EdgeMaps.

In [ ]:
dir(res.types.EdgeMap)

Algorithms are listed under `r.algos` and grouped by categories

In [ ]:
dir(res.algos)

In [ ]:
dir(res.algos.traversal)

## Example Usage

Let's see how to use metagraph by first constructing a graph from an edge list.

Begin with an input csv file representing an edge list and weights.

In [ ]:
data = """
Source,Destination,Weight
0,1,4
0,3,2
0,4,7
1,3,3
1,4,5
2,4,5
2,5,2
2,6,8
3,4,1
4,7,4
5,6,4
5,7,6
"""

Read in the csv file and convert to a Pandas DataFrame.

In [ ]:
import pandas as pd
import io
csv_file = io.StringIO(data)
df = pd.read_csv(csv_file)

This DataFrame represents a graph’s edges, but metagraph doesn’t know that yet.
To use the DataFrame within metagraph, we first need to convert it into
a Graph-like object.

A `PandasEdgeList` takes a DataFrame plus the labels of the columns representing source and destination nodes. With these, metagraph will know how to interpret the DataFrame as a Graph.

In [ ]:
g = res.wrappers.EdgeMap.PandasEdgeMap(df, 'Source', 'Destination', 'Weight', is_directed=False)
g.value

## Translate to other Graph formats

Because metagraph knows how to interpret `g` as a Graph, we can easily convert it other Graph formats.

Let's convert it to a NetworkX Graph.

In [ ]:
g2 = res.translate(g, res.wrappers.EdgeMap.NetworkXEdgeMap)
g2

The underlying object (in this case, a networkx Graph) is usually stored as the `.value` property.

We can verify that the edges are preserved correctly by inspecting the networkx Graph directly.

In [ ]:
g2.value.edges(data=True)

We can also convert `g` into an adjacency matrix representation using a GraphBLAS matrix.

The unweighted adjacency matrix has a weight value where an edge exists and is empty elsewhere.

In [ ]:
g3 = res.translate(g, res.types.EdgeMap.GrblasEdgeMapType)
g3

In [ ]:
g3.show()

We can also visualize the graph using functions found in the plugin libraries.

In [ ]:
import grblas
grblas.io.draw(g3.value)

## Inspect the steps required for translations

Rather than actually converting `g` into other formats, let's ask the system *how* it will do the conversion. Each conversion requires someone to write code to convert between the two formats. However, even if there isn't a direct translator between two formats, metagraph will find a path and take several translation steps as needed to perform the task.

The mechanism for viewing the plan is to invoke the translation from `r.plan.translate` rather than `r.translate`. Other than the additional `.plan`, the call signature is identical.

---
In this first example, there is a direct function which translates between `PandasEdgeMap` and `NetworkXEdgeMap`

In [ ]:
res.plan.translate(g, res.types.EdgeMap.NetworkXEdgeMapType)

---
In this next example, there is no direct function which convert `PandasEdgeMap` into a `GrblasEdgeMap`. Instead, we have to first convert to `NetworkXEdgeMap` and then to `ScipyEdgeMap` before finally arriving at our desired format.

While metagraph will do the conversion automatically, understanding the steps involved helps users plan for expected computation time and memory usage. If needed, they can also write a plugin to provide a direct translation path to save time. 

In [ ]:
res.plan.translate(g, res.types.EdgeMap.GrblasEdgeMapType)

## Algorithm Example #1: Triangle Count

Algorithms are described initially in an abstract definition. For triangle count, we take an `EdgeSet` and return an `int` indicating the number of unique triangles in the graph.

After the abstract definition is written, multiple concrete implementations are written to operate on concrete types.

Let's look at the signature and specific implementations available for triangle count.

In [ ]:
res.algos.cluster.triangle_count.signatures

We see that there are two implementations available. One takes a NetworkXEdgeSet. The other takes a ScipyEdgeSet.

---
Let's count the triangles with our different representations of `g`. We should get the same answer no matter which implementation is chosen.

In [ ]:
res.algos.cluster.triangle_count(g)

In [ ]:
res.algos.cluster.triangle_count.core_networkx(g2)

In [ ]:
res.plugins.core_networkx.algos.cluster.triangle_count(g)

In [ ]:
res.algos.cluster.triangle_count(g2)

---
Similar to how we can view the plan for translations, we can view the plan for algorithms.

Attempting to run triangle count with a PandasEdgeList will automatically convert to a NetworkX Graph, then run the algorithm.

In [ ]:
res.plan.algos.cluster.triangle_count(g)

---
In the next example, `g2` is already a NetworkX Graph, so the only translation needed is from an EdgeMap to an EdgeSet (i.e. dropping the weights).

In [ ]:
res.plan.algos.cluster.triangle_count(g2)

---
How do we make metagraph run the triangle_count algorithm written for scipy adjacency matrix?

Because it finds the networkx version first, it will choose that unless we start with a scipy matrix.

In [ ]:
g4 = res.translate(g2, res.types.EdgeMap.ScipyEdgeMapType)
res.plan.algos.cluster.triangle_count(g4)

---
Just to prove that it gives the same result, let's run it

In [ ]:
res.algos.cluster.triangle_count(g4)

## Algorithm Example #2: Pagerank

Let's look at the same pieces of information, but for pagerank. Pagerank takes a Graph and returns a NodeMap, indicating the rank value of each node in the graph.

First, let's verify the signature and the implementations available.

We see that there is only one implementation available, which takes a NetworkX Graph as input.

In [ ]:
res.algos.link_analysis.pagerank.signatures

---
Let's look at the steps required in the plan. Then let's perform the computation.

In [ ]:
res.plan.algos.link_analysis.pagerank(g)

In [ ]:
pr = res.algos.link_analysis.pagerank(g)
pr

The result is a PythonNodeMap. Its underlying object is just a dict, so we can view that easily.

In [ ]:
pr.value

Suppose we want to use the result in a numpy function. We could create the numpy array from the dict, but there is already a translator available to do that. Let's use it.

In [ ]:
pr_nicer = res.translate(pr, res.types.NodeMap.NumpyNodeMapType)
pr_nicer.value